In [1]:
'''Importing Relevant Libraries'''

import math 
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt
import quaternion as Quaternion
from matplotlib.ticker import ScalarFormatter

import pca

from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.signal import decimate, resample_poly, resample
from scipy.interpolate import PchipInterpolator

from ahrs.filters.madgwick import Madgwick #ahrs library requires git clone, do not pip install



In [7]:
gps1 = pd.read_csv("gps_1.csv") # Storing csv file into pandas dataframe
gps2 = pd.read_csv("gps_2.csv")

column_list = gps1.columns
print(gps1.columns)

gps1 = gps1.to_numpy() # Converting to Numpy file
gps2 = gps2.to_numpy()

gps1 = gps1[1:,:]
gps2 = gps2[1:,:]

print(gps1.shape)
print(gps2.shape)

Index(['ITOW', 'A_ECEFX', 'A_ECEFY', 'A_ECEFZ', 'A_ECEFMAG', 'A_LAT', 'A_LON',
       'A_H', 'A_HMSL', 'A_N', 'A_E', 'A_D', 'A_PACC', 'A_HACC', 'A_VACC',
       'B_ECEFX', 'B_ECEFY', 'B_ECEFZ', 'B_ECEFMAG', 'B_LAT', 'B_LON', 'B_H',
       'B_HMSL', 'B_N', 'B_E', 'B_D', 'B_PACC', 'B_HACC', 'B_VACC', 'C_ECEFX',
       'C_ECEFY', 'C_ECEFZ', 'C_ECEFMAG', 'C_LAT', 'C_LON', 'C_H', 'C_HMSL',
       'C_N', 'C_E', 'C_D', 'C_PACC', 'C_HACC', 'C_VACC'],
      dtype='object')
(58904, 43)
(108170, 43)


In [ ]:
stdev_gps_1 = np.std(gps1, axis = 0)
stdev_gps_2 = np.std(gps2, axis = 0)

print(len(stdev_gps_1))

print(stdev_gps_1)
print(stdev_gps_2)

output = ""
for i in range(1, 43):
    output += "Standard Deviation of "
    output = output + column_list[i] + ": " + str(stdev_gps_1[i])
    output += "\n"

print(output)


43
[3.40197238e+06 7.11071293e+00 1.27501628e+01 7.55955588e+00
 1.40701310e+01 4.95989424e-08 6.93343501e-08 1.40648369e+01
 1.40648398e+01 0.00000000e+00 0.00000000e+00 0.00000000e+00
 1.29120970e+00 3.55271368e-15 1.87947829e+00 8.10020594e+00
 2.34969205e+01 1.24490893e+01 2.53533176e+01 7.90219528e-08
 7.82095686e-08 2.53417430e+01 2.53417317e+01 9.68454808e+00
 1.08658601e+01 2.57894047e+01 1.56241452e+00 2.65728594e-02
 2.27583097e+00 4.23202260e+00 1.22734500e+01 9.77563780e+00
 1.37616617e+01 6.79768411e-08 4.51504304e-08 1.37616411e+01
 1.37618654e+01 7.80893946e+00 7.42846171e+00 1.66837348e+01
 7.35035965e-01 9.21310398e-04 1.08432048e+00]
[1.20228062e+07 5.74440029e+00 1.29128687e+01 9.13162890e+00
 1.56104453e+01 3.89576472e-08 4.86561249e-08 1.56100473e+01
 1.56103148e+01 0.00000000e+00 0.00000000e+00 0.00000000e+00
 5.40738767e-01 3.55271368e-15 8.38904962e-01 5.03746284e+00
 1.12760277e+01 9.10590916e+00 1.35401780e+01 4.77057072e-08
 5.23467709e-08 1.35427656e+01 1.35

In [ ]:
# Interpolation to estimate missing data
